In [12]:
# import numpy as np
# import pandas as pd
# import os
#
#
# from afinn import Afinn
#
# from collections import Counter
# from sklearn.feature_extraction.text import CountVectorizer
# import matplotlib.pyplot as plt
# import networkx as nx

import spacy
from nltk import sent_tokenize, word_tokenize
import itertools
import en_core_web_sm
import operator




In [13]:
nlp = en_core_web_sm.load()

In [27]:
def read_text(path):
    with open(path, encoding="utf-8") as f:
        text = f.read()
        text = text.replace('\r', ' ').replace('\n', ' ')\
            .replace("’", "'").replace("\"", "").replace("”", "").replace("“", "")
    return text

def spacy_NER(book, eval = False):
    """
    spacy_NER vrne seznam, v katerem so shranjene prepoznane identitete glede na posamezni stavek
    :param book: str
    :return: entity_dict (seznam slovarjev kot npr. [name, tag, start_pos, stop_pos, line_num. token_num])
    """

    # EVAL ---
    if eval == True:
        sentences, sentences_tag = conll_read()
        #print(len(sentences), "--------")
    else:
        sentences = sent_tokenize(book)
    sentences_NER = []
    sentences_NER_results = []
    # EVAL ---

    # 00 Pretvori knjigo v stavke:
    #sentences = sent_tokenize(book)
    entity_dict = []


    for line_num, line in enumerate(sentences):
        print(line_num)

        # EVAL ---
        words_list = word_tokenize(line)
        tags_list = ["O"] * len(words_list)
        sentences_NER.append(words_list)
        # EVAL ---

        entities = nlp(line)
        for word in entities.ents:
            #print(word.text,word.label_, word.start, word.end)
            info_dict = {}
            info_dict["name"] = word.text
            info_dict["tag"] = word.label_
            info_dict["start_pos"] = word.start
            info_dict["stop_pos"] = word.end
            info_dict["line_num"] = line_num
            entity_dict.append(info_dict)

            # EVAL: ---
            names = word.text.replace("'s", "")
            names = names.split(" ")
            ff_flag = False
            if word.label_ == "PERSON":
                for name in names:
                    try:
                        if ff_flag == False:
                            idx = words_list.index(name)
                            tags_list[idx] = "B-PER"
                            ff_flag = True
                        else:
                            idx = words_list.index(name)
                            tags_list[idx] = "I-PER"
                    except:



                        print("Not found: ", name)
            # EVAL ---
        sentences_NER_results.append(tags_list)
    return entity_dict, sentences_NER, sentences_NER_results


def get_names_from_NER(entity_dict):
    """
    get_names_from_NER sprejme entity_dict in vrne urejen seznam terk ("ime", št_zaznano)
    :param entity_dict: dict (seznam dictov)
    :return: unique_names: list
    """
    unique_names = {}

    for entity in entity_dict:
        print(entity)
        if entity["tag"] == "PERSON":
            if entity["name"] not in unique_names:
                unique_names[entity["name"]] = 1
            else:
                unique_names[entity["name"]] += 1
    unique_names = sorted(unique_names.items(), key=operator.itemgetter(1),reverse=True)

    return unique_names

def conll_read():
    with open("conll2003/valid.txt") as f:
        lines = f.readlines()
    sentences = []
    sentences_tag = []
    tmp = ""
    tmp_tag = ""
    tags = []
    for i in range(len(lines)):

        line = lines[i]
        line = line.split(" ")
        text_word = line[0]

        if text_word[0] != "-":
            if text_word == "\n":
                # print(tmp)
                # print(tmp_tag)
                # print("-------")
                sentences.append(tmp)
                tmp = ""
                #sentences_tag.append(tmp_tag)
                #tmp_tag = ""
                sentences_tag.append(tags)
                tags = []

            elif text_word == "(" or text_word == ")" or text_word == '"' or text_word == ':':
                pass
            else:
                if text_word[0] != "." and text_word[0] != "," and text_word[0] != ":" and text_word[0] != ";"\
                        and text_word[0] != "!" and text_word[0] != "?" and text_word[0] != ")" and text_word[0] != "'":   #or text_word[0] != "," or text_word[0] != "(" or text_word[0] != ")":
                    tmp = tmp + " " + text_word
                    line[3] = line[3].replace("\n", "")

                    if line[3] == "B-PER" or line[3] == "I-PER":
                        #line[3] = "B-PER"
                        #tmp_tag = tmp_tag + " " + line[3]
                        tags.append(line[3])
                    else:
                        line[3] = "O"
                        #tmp_tag = tmp_tag + " " + line[3]
                        tags.append(line[3])
                else:
                    tmp = tmp +  text_word
                    line[3] = "O"
                    tags.append(line[3])
    return sentences, sentences_tag



In [28]:
book = read_text('../../data/books/ASongOfIceAndFire/AGOT/chapters/Bran_1_1.txt')
book = "George Washington went to Washington."
entity_dict, sentences_NER, sentences_NER_results = spacy_NER(book, True)
unique_names = get_names_from_NER(entity_dict)

for (name, num) in unique_names:
    if True: #num > 1:
        print(name, num)





0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [29]:
# Kontrola podatkov
sentences_org, sentences_org_tag = conll_read()
# for x in range(len(sentences_NER)):
#     print("-----------", x)
#     #print(len(sentences_org_tag[x]))
#     print(len(sentences_NER[x]))
#     print(len(sentences_NER_results[x]))
#
#     #print(sentences_org[x])    # 22 26
#     print(sentences_NER[x])
#     print(sentences_org_tag[x])
#     print(sentences_NER_results[x])

In [30]:
from nervaluate import Evaluator
from nltk.tokenize import sent_tokenize

In [31]:
true = sentences_org_tag
pred = sentences_NER_results
evaluator = Evaluator(true, pred, tags=['PER'], loader="list")

results, results_by_tag = evaluator.evaluate()
print(results)

{'ent_type': {'correct': 1103, 'incorrect': 0, 'partial': 0, 'missed': 739, 'spurious': 167, 'possible': 1842, 'actual': 1270, 'precision': 0.868503937007874, 'recall': 0.5988056460369164, 'f1': 0.7088688946015425}, 'partial': {'correct': 887, 'incorrect': 0, 'partial': 216, 'missed': 739, 'spurious': 167, 'possible': 1842, 'actual': 1270, 'precision': 0.7834645669291339, 'recall': 0.5401737242128122, 'f1': 0.6394601542416453}, 'strict': {'correct': 887, 'incorrect': 216, 'partial': 0, 'missed': 739, 'spurious': 167, 'possible': 1842, 'actual': 1270, 'precision': 0.6984251968503937, 'recall': 0.4815418023887079, 'f1': 0.5700514138817482}, 'exact': {'correct': 887, 'incorrect': 216, 'partial': 0, 'missed': 739, 'spurious': 167, 'possible': 1842, 'actual': 1270, 'precision': 0.6984251968503937, 'recall': 0.4815418023887079, 'f1': 0.5700514138817482}}
